In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from transcriber.data.utils import get_resource

data_fol = get_resource("data_fol")
course_fol = data_fol / "course"
videos_fol = course_fol / "videos"
output_fol = course_fol / "transcripts"
tmp_audio = get_resource("tmp_audio")

In [ ]:
# create the transcripts folder
from transcriber.data.utils import check_create_fol


check_create_fol(output_fol)

In [ ]:
# delete the output folder and the contents
# import shutil
# shutil.rmtree(output_fol)

In [ ]:
import whisper

model_type = "medium"
model = whisper.load_model(model_type)

In [ ]:
from pathlib import Path
from pydub import AudioSegment


def extract_audio(video_fp: Path, audio_fp: Path, audio_format: str = "mp3") -> None:
    video_format = video_fp.suffix[1:]
    video = AudioSegment.from_file(video_fp, format=video_format)
    audio = video.set_channels(1).set_frame_rate(16000).set_sample_width(2)
    audio.export(audio_fp, format=audio_format)

In [ ]:
from whisper import Whisper


def extract_text(audio_fp: Path, model: Whisper) -> str:
    result = model.transcribe(str(audio_fp))
    return result["text"]

In [ ]:
# iterate the topics in the course
for topic_fol in videos_fol.iterdir():
    if not topic_fol.is_dir():
        continue
    topic_name = topic_fol.name
    print(topic_name)
    # create the topic folder in the output folder
    topic_output_fol = output_fol / topic_name
    check_create_fol(topic_output_fol)

    # iterate the videos in the topic
    for video_fp in topic_fol.iterdir():
        video_name = video_fp.stem
        print("  ", video_name)
        # build the output txt file name
        transcript_fp = topic_output_fol / f"{video_name}.txt"
        # if the transcript file already exists, skip
        if transcript_fp.exists():
            continue
        # extract the audio from the video
        extract_audio(video_fp, tmp_audio)
        # extract the text from the audio
        text = extract_text(tmp_audio, model)
        # write the text to the transcript file
        transcript_fp.write_text(text)
    #     break
    # break